<a href="https://colab.research.google.com/github/pratamastan/bangkit-capstone0143/blob/main/NLP%20model%20proto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
data1=pd.read_csv("/content/drive/MyDrive/Quora question pair dataset/train complete indo.csv")

In [ ]:
data1=data1.dropna()

In [ ]:
data1.isnull().values.any()

False

In [ ]:
data1.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,Apa panduan langkah demi langkah untuk berinve...,Apa panduan langkah demi langkah untuk berinve...,0
1,1,3,4,Bagaimana kisah Berlian Kohinoor (Koh-i-Noor)?,Apa yang akan terjadi jika pemerintah India me...,0
2,2,5,6,Bagaimana cara meningkatkan kecepatan koneksi ...,Bagaimana kecepatan Internet dapat ditingkatka...,0
3,3,7,8,Mengapa saya secara mental sangat kesepian? Ba...,Temukan sisanya ketika [math] 23 ^ {24} [/ mat...,0
4,4,9,10,Manakah yang larut dalam air seperti gula pasi...,Ikan apa yang akan bertahan hidup di air asin?,0


In [ ]:
data1.question1.str.len().max()

717

In [ ]:
data1.question2.str.len().max()

1395

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data1[["question1", "question2"]], data1["is_duplicate"], test_size=0.2, random_state=42)

In [ ]:
xt=X_train.to_numpy()
yt=y_train.to_numpy()

xte=X_test.to_numpy()
yte=y_test.to_numpy()

Q1_train = X_train["question1"].to_numpy()
Q2_train = X_train["question2"].to_numpy()
Q1_test = X_test["question1"].to_numpy()
Q2_test = X_test["question2"].to_numpy()

In [ ]:
Q1_train

array(['Apa sajakah cerita pendeknya?',
       'Apa jawaban terbaik untuk pertanyaan HR, "Anda berasal dari latar belakang ECE, jadi mengapa Anda ingin bergabung dengan perusahaan IT"?',
       'Bagaimana cara membulatkan 1,44 ke persen terdekat?', ...,
       'Siapa yang saat ini memenangkan pemilihan presiden?',
       'Apa sajakah film Telugu / Tamil / Hindi yang pemeran utamanya menjadi mafia atau tidak?',
       'Berapa peringkat yang dipotong untuk perguruan tinggi Kedokteran Rohtak dalam kuota negara bagian?'],
      dtype=object)

In [ ]:
data1["question1"][0]

'Apa panduan langkah demi langkah untuk berinvestasi di pasar saham di India?'

In [ ]:
data1[["question1","question2"]]

,question1,question2
0,Apa panduan langkah demi langkah untuk berinve...,Apa panduan langkah demi langkah untuk berinve...
1,Bagaimana kisah Berlian Kohinoor (Koh-i-Noor)?,Apa yang akan terjadi jika pemerintah India me...
2,Bagaimana cara meningkatkan kecepatan koneksi ...,Bagaimana kecepatan Internet dapat ditingkatka...
3,Mengapa saya secara mental sangat kesepian? Ba...,Temukan sisanya ketika [math] 23 ^ {24} [/ mat...
4,Manakah yang larut dalam air seperti gula pasi...,Ikan apa yang akan bertahan hidup di air asin?
...,...,...
404285,Ada berapa kata kunci dalam bahasa pemrograman...,Ada berapa kata kunci dalam Bahasa Pemrograman...
404286,Apakah Anda percaya ada kehidupan setelah kema...,Benarkah ada kehidupan setelah kematian?
404287,Apa satu koin?,Koin apa ini?
404288,Berapa perkiraan biaya hidup tahunan selama be...,Saya mengalami sedikit masalah rambut rontok t...


In [ ]:
embed = hub.KerasLayer("https://tfhub.dev/google/nnlm-id-dim128-with-normalization/2",input_shape=[], dtype=tf.string)
embed.trainable=True

In [ ]:
xt1=embed(Q1_train)
xt2=embed(Q2_train)
xte1=embed(Q1_test)
xte2=embed(Q2_test)

In [ ]:
xt1

<tf.Tensor: shape=(323430, 128), dtype=float32, numpy=
array([[ 0.03792422, -0.08203722, -0.03952134, ..., -0.03751129,
        -0.13026127, -0.21294448],
       [ 0.16049062,  0.05221158, -0.03756306, ..., -0.06843793,
        -0.16083518,  0.00526717],
       [ 0.00541904,  0.03682816, -0.10739181, ..., -0.05703003,
        -0.12414219, -0.05216019],
       ...,
       [ 0.17406467,  0.06946962, -0.02178098, ..., -0.05727712,
        -0.17851247, -0.00691927],
       [ 0.14288372, -0.16400667,  0.03347619, ...,  0.04388543,
        -0.12863201, -0.02391369],
       [-0.02106932, -0.22670205, -0.11702876, ..., -0.13742013,
        -0.11424292,  0.08969257]], dtype=float32)>

In [ ]:
#ini model 1 epoch sekitar 2 menit
model=tf.keras.Sequential([tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                           tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                           tf.keras.layers.Dense(200,activation='relu'),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.BatchNormalization(),
                           tf.keras.layers.Dense(200,activation='relu'),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.BatchNormalization(),
                           tf.keras.layers.Dense(200,activation='relu'),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.BatchNormalization(),
                           tf.keras.layers.Dense(200,activation='relu'),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.BatchNormalization(),
                           tf.keras.layers.Dense(1, activation="sigmoid")
])
#model.build((1,2,128))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 2, 128)            98816     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 200)               13000     
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 200)               800       
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0

In [ ]:
#ini model running timenya lama, 1 epoch sekitar 20 menit
ques1=tf.keras.Input(shape=(), dtype=tf.string)
ques2=tf.keras.Input(shape=(), dtype=tf.string)

qu1=embed(ques1)
qu2=embed(ques2)

q1=tf.expand_dims(qu1, axis=1)
q2=tf.expand_dims(qu2, axis=1)
q1=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(q1)
q2=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(q2)

attention=tf.keras.layers.Attention()([q1,q2])
attention=tf.keras.layers.Flatten()(attention)
attention=tf.keras.layers.Dense(300, activation="relu")(attention)
#attention=tf.keras.layers.Reshape((2,128))(attention)

#merged = tf.keras.layers.Add([q1,attention])
merged = tf.keras.layers.Flatten()(attention)
merged = tf.keras.layers.Dense(200,activation='relu')(merged)
merged = tf.keras.layers.Dropout(0.2)(merged)
merged = tf.keras.layers.BatchNormalization()(merged)
merged = tf.keras.layers.Dense(200,activation='relu')(merged)
merged = tf.keras.layers.Dropout(0.2)(merged)
merged = tf.keras.layers.BatchNormalization()(merged)
merged = tf.keras.layers.Dense(200,activation='relu')(merged)
merged = tf.keras.layers.Dropout(0.2)(merged)
merged = tf.keras.layers.BatchNormalization()(merged)
merged = tf.keras.layers.Dense(200,activation='relu')(merged)
merged = tf.keras.layers.Dropout(0.2)(merged)
merged = tf.keras.layers.BatchNormalization()(merged)

is_duplicate = tf.keras.layers.Dense(1, activation="sigmoid")(merged)
model1 = tf.keras.models.Model(inputs=[ques1,ques2], outputs=is_duplicate)
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        (None, 128)          112461824   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
tf.expand_dims (TFOpLambda)     (None, 1, 128)       0           keras_layer[0][0]            

In [ ]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history1 = model1.fit([Q1_train, Q2_train],
                    yt,
                    epochs=5,
                    validation_data=([Q1_test,Q2_test], yte),
                    verbose=2)

Epoch 1/5
10108/10108 - 1098s - loss: 0.5601 - accuracy: 0.7142 - val_loss: 0.5219 - val_accuracy: 0.7394
Epoch 2/5
10108/10108 - 1059s - loss: 0.5041 - accuracy: 0.7527 - val_loss: 0.5153 - val_accuracy: 0.7471
Epoch 3/5
10108/10108 - 1060s - loss: 0.4687 - accuracy: 0.7760 - val_loss: 0.5100 - val_accuracy: 0.7544
Epoch 4/5
10108/10108 - 1055s - loss: 0.4320 - accuracy: 0.7981 - val_loss: 0.5163 - val_accuracy: 0.7551
Epoch 5/5
10108/10108 - 1057s - loss: 0.3958 - accuracy: 0.8192 - val_loss: 0.5457 - val_accuracy: 0.7532


In [ ]:
history = model.fit(np.stack((xt1, xt2), axis=1),
                    yt,
                    epochs=5,
                    validation_data=(np.stack((xte1, xte2), axis=1), yte),
                    verbose=2)

Epoch 1/5
10108/10108 - 127s - loss: 0.5713 - accuracy: 0.7028 - val_loss: 0.5203 - val_accuracy: 0.7332
Epoch 2/5
10108/10108 - 89s - loss: 0.4990 - accuracy: 0.7478 - val_loss: 0.4760 - val_accuracy: 0.7597
Epoch 3/5
10108/10108 - 87s - loss: 0.4668 - accuracy: 0.7676 - val_loss: 0.4600 - val_accuracy: 0.7681
Epoch 4/5
10108/10108 - 86s - loss: 0.4468 - accuracy: 0.7809 - val_loss: 0.4450 - val_accuracy: 0.7805
Epoch 5/5
10108/10108 - 86s - loss: 0.4299 - accuracy: 0.7923 - val_loss: 0.4362 - val_accuracy: 0.7875


In [ ]:
a=embed(["saya memukul kursi"])

In [ ]:
a2=embed(["kursi dipukul oleh saya"])

In [ ]:
prediction1 = model1.predict([np.asarray(["dia memainkan gitar"]),np.asarray(["gitar dimainkan oleh dia"])])
print(prediction1)

[[0.3453182]]


In [ ]:
np.stack((a,a2),axis=1)

array([[[-0.02527324,  0.13880529, -0.0091286 ,  0.16133781,
          0.16281272, -0.09192153, -0.05400613,  0.01175207,
         -0.00157341,  0.11544802,  0.04184154, -0.02567816,
          0.02595361,  0.03100132,  0.08969943,  0.16123143,
         -0.05234122,  0.04622787,  0.02177284,  0.04983375,
          0.07719582,  0.01804273, -0.05729916, -0.08064298,
          0.03315999,  0.03386731, -0.08410315,  0.11704644,
          0.02580629, -0.11809724,  0.00287541,  0.03695798,
          0.01039357,  0.0519742 , -0.20322444,  0.08522063,
          0.05121319, -0.03870725, -0.07432587,  0.01950176,
          0.12678719,  0.01796092,  0.14816546,  0.02209316,
         -0.04718846, -0.10268682, -0.07144903,  0.01819091,
          0.17900167,  0.1234579 , -0.0631758 ,  0.0115751 ,
         -0.09392782, -0.13392727, -0.07532788,  0.18672585,
         -0.05976349, -0.08942653,  0.10088743,  0.02016073,
          0.00038054, -0.04717613,  0.14853036, -0.02136726,
         -0.06254555, -0

In [ ]:
prediction = model.predict(np.stack((a,a2),axis=1))
print(prediction)

[[0.6698727]]
